# 📅 Day 32 – Fine-Tuning the LLaMA Model

## 🔄 Continuation of Day 31

This notebook builds directly on the work completed in **Day 31**, where we:

- Loaded the **Appliances** subset from the 2023 Amazon Reviews dataset.
- Parsed reviews into structured `Item` objects, each with a `prompt` and numeric `price`.
- Explored the dataset through token count, price distributions, and correlation plots.
- Split the data into training (25,000 items) and test sets (2,000 items).
- Packaged and pushed the dataset to Hugging Face as `david125tran/lite-data`.
- Saved local `.pkl` files for quick reuse: `train_lite.pkl`, `test_lite.pkl`.

---

## 🎯 Objectives for Today

- Load the training and test data generated yesterday.
- Prepare the dataset for fine-tuning a **pretrained LLaMA model**.
- Fine-tune the model to predict product prices based on textual prompts.
- Evaluate model performance on the held-out test set.
- Optionally: push the fine-tuned model to Hugging Face for later use.

---

## 📦 Input Dependencies

- `train_lite.pkl` and `test_lite.pkl` files created during Day 31.
- Tokenizer and base model must match the formatting used to generate the prompts.
- Assumes access to Hugging Face API credentials for model usage and uploads.

---

## ✅ Key Outcomes

- A trained regression-capable language model that maps product descriptions to price.
- A consistent workflow for prompt → price learning using real-world e-commerce text.
- A pushable model checkpoint and reproducible pipeline for future expansion.



In [1]:
# pip installs

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# imports
# With much thanks to Islam S. for identifying that there was a missing import!

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [4]:
# ================================
# 🔧 Constants and Config Setup
# ================================

# Base model from Hugging Face to fine-tune (LLaMA 3.1 8B parameter base model)
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"

# Name for the project you're training — used for tracking, naming, and logging
PROJECT_NAME = "lite-data"

# Your Hugging Face username — used for saving datasets and models under your namespace
HF_USER = "david125tran"

# ==================================
# 📦 Dataset Configuration
# ==================================

# This is the path (on Hugging Face) to your dataset, used to load training data
DATASET_NAME = f"{HF_USER}/lite-data"

# Limit on the number of tokens per example — important for managing memory usage
MAX_SEQUENCE_LENGTH = 182

# ===================================
# 💾 Model Naming & Version Control
# ===================================

# Generate a unique timestamp for this training run
RUN_NAME = f"{datetime.now():%Y-%m-%d_%H.%M.%S}"

# Combine project and timestamp to name this specific run
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"

# Full Hugging Face path where the trained model will be saved
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

# ===================================
# 🧠 LoRA (Low-Rank Adaptation) Settings
# ===================================

# LoRA rank: the dimensionality of the trainable adapter matrix
LORA_R = 32

# LoRA alpha: scaling factor applied to the LoRA weights (often 2 * R)
LORA_ALPHA = 64

# LoRA target modules: which layers of the transformer to fine-tune
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

# Dropout rate applied during LoRA training to prevent overfitting
LORA_DROPOUT = 0.1

# Whether to use 4-bit quantization for loading the base model — saves memory
QUANT_4_BIT = True

# ===================================
# 🏋️ Training Hyperparameters
# ===================================

# Number of passes through the training dataset
EPOCHS = 1  # 1 is often enough for targeted tasks

# Number of training samples processed per step (GPU memory dependent)
BATCH_SIZE = 16  # A100 GPUs can typically handle up to 16 here.  Maybe use 1 for a T4 box

# Number of steps to accumulate gradients before updating weights
GRADIENT_ACCUMULATION_STEPS = 1

# Initial learning rate for the optimizer — critical for model convergence
LEARNING_RATE = 1e-4

# Type of learning rate decay schedule to use over training
LR_SCHEDULER_TYPE = 'cosine'

# Proportion of training to spend slowly ramping up the learning rate
WARMUP_RATIO = 0.03

# Optimizer used for gradient descent — AdamW with memory paging for efficiency
OPTIMIZER = "paged_adamw_32bit"

# ===================================
# 🧾 Administrative Training Settings
# ===================================

# Frequency (in steps) to print training logs or checkpoint stats
STEPS = 50

# Frequency (in steps) to save model checkpoints to Hugging Face Hub
SAVE_STEPS = 2000  # More frequent saves for easier monitoring & rollback

# Whether to enable logging to Weights & Biases (W&B) dashboard
LOG_TO_WANDB = True

# Enable inline plots for training visualization in notebooks
%matplotlib inline


In [5]:
HUB_MODEL_NAME

'david125tran/lite-data-2025-07-05_16.26.10'

# More on Optimizers

https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#optimizer-choice

The most common is Adam or AdamW (Adam with Weight Decay).  
Adam achieves good convergence by storing the rolling average of the previous gradients; however, it adds an additional memory footprint of the order of the number of model parameters.


### Log in to HuggingFace and Weights & Biases

If you don't already have a HuggingFace account, visit https://huggingface.co to sign up and create a token.

Then select the Secrets for this Notebook by clicking on the key icon in the left, and add a new secret called `HF_TOKEN` with the value as your token.

Repeat this for weightsandbiases at https://wandb.ai and add a secret called `WANDB_API_KEY`

In [6]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [7]:
# Log in to Weights & Biases
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: Currently logged in as: david125tran (david125tran-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/9.84M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/780k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
# Train on less data to speed it up
train = train.select(range(20000))

In [10]:
train[0]

{'text': 'How much does this cost to the nearest dollar?\n\nand Replacement Range Cooktop Drip Pans fit GE, Hotpoint - Two 6 Inch and Two 8 Inch Pans (4 pieces)\nContents 2 x (6 inches) and 2 x (8 inches) bowls, 4 drip bowls total Compatibility This replacement kit works with GE, Hotpoint, Moffat, Monogram (GE), Profile (GE), RCA (GE), and Roper models prior to 1996. replaces 65975, replaces and 65974, 770169 Premium quality Drip bowls are made of durable high-quality material. It features a chrome finish, well-tested by the manufacturer. Durable, stick-free, easy to clean, and dishwasher safe. Ensure long-lasting and effective performance Easy to install Shut off electrical power, tilt the coil\n\nPrice is $12.00',
 'price': 11.99}

## train[0]:

{'text': 'How much does this cost to the nearest dollar?\n\nand Replacement Range Cooktop Drip Pans fit GE, Hotpoint - Two 6 Inch and Two 8 Inch Pans (4 pieces)\nContents 2 x (6 inches) and 2 x (8 inches) bowls, 4 drip bowls total Compatibility This replacement kit works with GE, Hotpoint, Moffat, Monogram (GE), Profile (GE), RCA (GE), and Roper models prior to 1996. replaces 65975, replaces and 65974, 770169 Premium quality Drip bowls are made of durable high-quality material. It features a chrome finish, well-tested by the manufacturer. Durable, stick-free, easy to clean, and dishwasher safe. Ensure long-lasting and effective performance Easy to install Shut off electrical power, tilt the coil\n\n**Price is $12.00'**,
 **'price': 11.99**}
  
 The model will then start trying to predict the price

In [11]:
if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Now load the Tokenizer and Model

The model is "quantized" - we are reducing the precision to 4 bits.

In [12]:
# pick the right quantization

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

### 🧮 Quantization Configuration (4-bit vs 8-bit)

This block sets up the quantization settings using `BitsAndBytesConfig` from the `bitsandbytes` library. It enables loading large language models efficiently by reducing precision and memory usage.

```python
if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,                      # Enable 4-bit loading
        bnb_4bit_use_double_quant=True,         # Use double quantization (better compression)
        bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16 for computation
        bnb_4bit_quant_type="nf4"               # Use "nf4" quantization (normal float 4)
    )
else:
    quant_config = BitsAndBytesConfig(  
        load_in_8bit=True,                      # Enable 8-bit loading
        bnb_8bit_compute_dtype=torch.bfloat16   # Use bfloat16 for math
    )
```
### 🔹 Key Points:
- 4-bit quantization (NF4) reduces memory usage significantly.
- 8-bit quantization is slightly more accurate but requires more memory.
- Both options allow loading large models (like LLaMA or Mistral) on GPUs with limited VRAM (e.g., 8–16GB).
- bfloat16 is used for computation for improved performance on modern GPUs.
- Use this configuration when loading models with:
```
AutoModelForCausalLM.from_pretrained(..., quantization_config=quant_config)
```

In [13]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Memory footprint: 5591.5 MB


# Data Collator

It's important that we ensure during Training that we are not trying to train the model to predict the description of products; only their price.

We need to tell the trainer that everything up to "Price is $" is there to give context to the model to predict the next token, but does not need to be learned.

The trainer needs to teach the model to predict the token(s) after "Price is $".

There is a complicated way to do this by setting Masks, but luckily HuggingFace provides a super simple helper class to take care of this for us.

In [14]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# AND NOW

## We set up the configuration for Training

We need to create 2 objects:

A LoraConfig object with our hyperparameters for LoRA

An SFTConfig with our overall Training parameters

In [15]:
# First, specify the configuration parameters for LoRA

lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# Next, specify the general configuration parameters for training

train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True
)

# And now, the Supervised Fine Tuning Trainer will carry out the fine-tuning
# Given these 2 sets of configuration parameters
# The latest version of trl is showing a warning about labels - please ignore this warning
# But let me know if you don't see good training results (loss coming down).

fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator
  )

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

## In the next cell, we kick off fine-tuning!

This will run for some time, uploading to the hub every SAVE_STEPS steps.

After some time, Google might stop your colab. For people on free plans, it can happen whenever Google is low on resources. For anyone on paid plans, they can give you up to 24 hours, but there's no guarantee.

If your server is stopped, you can follow my colab here to resume from your last save:

https://colab.research.google.com/drive/1qGTDVIas_Vwoby4UVi2vwsU0tHXy8OMO#scrollTo=R_O04fKxMMT-

I've saved this colab with my final run in the output so you can see the example. The trick is that I needed to set `is_trainable=True` when loading the fine_tuned model.

### Anyway, with that in mind, let's kick this off!

In [16]:
# Fine-tune!
fine_tuning.train()

# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=False)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


Step,Training Loss
50,1.641500
100,1.387700
150,1.367200
200,1.355400
250,1.359900
300,1.333000
350,1.338700
400,1.315100
450,1.310900
500,1.305000


wandb: Adding directory to artifact (./lite-data-2025-07-05_16.26.10/checkpoint-1250)... Done. 0.7s


README.md:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/109M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved to the hub: lite-data-2025-07-05_16.26.10


In [17]:
if LOG_TO_WANDB:
  wandb.finish()

train/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇███
train/grad_norm,▇▇▁▁▄▃▁▂▄▃▄▁▄▅▁▁█▃▃▇▄▄▃▃▃
train/learning_rate,████▇▇▇▇▆▆▅▅▄▄▄▃▃▂▂▂▁▁▁▁▁
train/loss,█▄▃▃▃▃▃▂▂▂▂▂▁▂▂▁▁▁▂▂▁▂▁▁▁
total_flos,1.6054839894461645e+17
train/epoch,1
train/global_step,1250
train/grad_norm,1.79377
train/learning_rate,0
train/loss,1.2309
